# Azure AI Search र NVIDIA NIM र LlamaIndex एकीकरण

यस नोटबुकमा, हामी NVIDIA का AI मोडेलहरू र LlamaIndex प्रयोग गरेर शक्तिशाली Retrieval-Augmented Generation (RAG) पाइपलाइन कसरी निर्माण गर्ने देखाउनेछौं। हामी NVIDIA का LLMs र embeddings प्रयोग गर्नेछौं, तिनीहरूलाई Azure AI Search सँग एकीकृत गर्नेछौं जसलाई भेक्टर स्टोरको रूपमा प्रयोग गरिन्छ, र RAG प्रदर्शन गरेर खोजको गुणस्तर र दक्षता सुधार गर्नेछौं।

## फाइदाहरू
- **स्केलेबिलिटी**: NVIDIA का ठूला भाषा मोडेलहरू र Azure AI Search प्रयोग गरेर स्केलेबल र कुशल पुनःप्राप्ति सुनिश्चित गर्नुहोस्।
- **खर्च दक्षता**: भेक्टर स्टोरेज र हाइब्रिड खोज प्रविधिहरू प्रयोग गरेर खोज र पुनःप्राप्ति अनुकूलित गर्नुहोस्।
- **उच्च प्रदर्शन**: शक्तिशाली LLMs लाई भेक्टराइज्ड खोजसँग संयोजन गरेर छिटो र सटीक प्रतिक्रिया प्राप्त गर्नुहोस्।
- **गुणस्तर**: सम्बन्धित पुनःप्राप्त दस्तावेजहरू प्रयोग गरेर LLM प्रतिक्रियाहरूलाई आधारभूत बनाउँदै उच्च खोज गुणस्तर कायम गर्नुहोस्।

## आवश्यकताहरू
- 🐍 Python 3.9 वा उच्च संस्करण
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API Key NVIDIA का LLMs र Embeddings मा पहुँचको लागि NVIDIA NIM माइक्रोसर्भिसहरू मार्फत

## समेटिएका विशेषताहरू
- ✅ NVIDIA LLM एकीकरण (हामी [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe) प्रयोग गर्नेछौं)
- ✅ NVIDIA Embeddings (हामी [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5) प्रयोग गर्नेछौं)
- ✅ Azure AI Search उन्नत पुनःप्राप्ति मोडहरू
- ✅ LlamaIndex प्रयोग गरेर दस्तावेज अनुक्रमण
- ✅ Azure AI Search र LlamaIndex सँग NVIDIA LLMs प्रयोग गरेर RAG

अब सुरु गरौं!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## स्थापना र आवश्यकताहरू  
Python संस्करण >3.10 प्रयोग गरेर Python वातावरण सिर्जना गर्नुहोस्।  

## सुरु गर्दै!  


NVIDIA AI Foundation मोडेलहरू प्रयोग गर्नको लागि तपाईंलाई `NVIDIA_API_KEY` आवश्यक छ:
1) [NVIDIA](https://build.nvidia.com/explore/discover) मा निःशुल्क खाता बनाउनुहोस्।
2) आफ्नो रोजाइको मोडेलमा क्लिक गर्नुहोस्।
3) Input अन्तर्गत, Python ट्याब चयन गर्नुहोस्, त्यसपछि **Get API Key** मा क्लिक गर्नुहोस् र **Generate Key** मा क्लिक गर्नुहोस्।
4) उत्पन्न गरिएको key लाई NVIDIA_API_KEY को रूपमा प्रतिलिपि गर्नुहोस् र सुरक्षित राख्नुहोस्। त्यसपछि, तपाईंले endpoints मा पहुँच पाउनुहुनेछ।


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAG उदाहरण LLM र Embedding प्रयोग गर्दै
### १) LLM सुरु गर्नुहोस्
`llama-index-llms-nvidia`, जसलाई NVIDIA को LLM कनेक्टर पनि भनिन्छ, तपाईंलाई NVIDIA API क्याटलगमा उपलब्ध उपयुक्त मोडेलहरूसँग जडान गर्न र तिनीहरूबाट उत्पादन गर्न अनुमति दिन्छ। च्याट कम्प्लिशन मोडेलहरूको सूची यहाँ हेर्नुहोस्: https://build.nvidia.com/search?term=Text-to-Text

यहाँ हामी **mixtral-8x7b-instruct-v0.1** प्रयोग गर्नेछौं।


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) एम्बेडिङ सुरु गर्नुहोस्  
`llama-index-embeddings-nvidia`, जसलाई NVIDIA को Embeddings कनेक्टर पनि भनिन्छ, तपाईंलाई NVIDIA API क्याटलगमा उपलब्ध उपयुक्त मोडेलहरूसँग जडान गर्न र तिनीहरूबाट उत्पादन गर्न अनुमति दिन्छ। हामीले `nvidia/nv-embedqa-e5-v5` लाई एम्बेडिङ मोडेलको रूपमा चयन गरेका छौं। यहाँ पाठ एम्बेडिङ मोडेलहरूको सूची हेर्नुहोस्: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding  


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### ५) आफ्नो डाटामा प्रश्न सोध्नको लागि क्वेरी इन्जिन बनाउनुहोस्

यहाँ Azure AI Search मा शुद्ध भेक्टर खोज प्रयोग गरेर र हाम्रो LLM (Phi-3.5-MOE) मा प्रतिक्रिया आधार बनाउँदै गरिएको एक क्वेरी छ।


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

यहाँ Azure AI खोजमा हाइब्रिड खोज प्रयोग गर्ने एक प्रश्न छ।


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### भेक्टर खोज विश्लेषण
LLM को प्रतिक्रिया स्रोत पाठमा उल्लेखित रूसको स्टक बजारसँग सम्बन्धित मुख्य आर्थिक परिणामहरूलाई सही रूपमा समेट्छ। विशेष रूपमा, यसले बताउँछ कि रूसी स्टक बजारमा ठूलो गिरावट आयो, जसले यसको मूल्यको ४०% गुमायो, र चलिरहेको परिस्थितिका कारण व्यापार स्थगित गरियो। यो प्रतिक्रिया स्रोतमा प्रदान गरिएको जानकारीसँग राम्रोसँग मेल खान्छ, जसले देखाउँछ कि LLM ले रूसको कार्यहरू र लगाइएको प्रतिबन्धहरूको परिणामस्वरूप स्टक बजारमा परेको प्रभावलाई सही रूपमा पहिचान र संक्षेपमा प्रस्तुत गरेको छ।

#### स्रोत नोड्स टिप्पणी
स्रोत नोड्सले अन्तर्राष्ट्रिय प्रतिबन्धका कारण रूसले सामना गरेको आर्थिक परिणामहरूको विस्तृत विवरण प्रदान गर्दछ। पाठले उल्लेख गर्दछ कि रूसी स्टक बजारले आफ्नो मूल्यको ४०% गुमायो, र व्यापार स्थगित गरियो। साथै, यसले अन्य आर्थिक प्रभावहरू जस्तै रूबलको अवमूल्यन र रूसको अर्थतन्त्रको व्यापक अलगावलाई पनि उल्लेख गर्दछ। LLM को प्रतिक्रियाले यी नोड्सबाट महत्वपूर्ण बिन्दुहरूलाई प्रभावकारी रूपमा संक्षेपमा प्रस्तुत गरेको छ, विशेष गरी सोधिएको प्रश्न अनुसार स्टक बजारको प्रभावमा केन्द्रित हुँदै।


अब, एउटा प्रश्नलाई हेरौं जहाँ Hybrid Search ले राम्रो आधारभूत उत्तर प्रदान गर्दैन:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### हाइब्रिड खोज: LLM प्रतिक्रिया विश्लेषण
हाइब्रिड खोजको उदाहरणमा LLM प्रतिक्रिया देखाउँछ कि दिइएको सन्दर्भले रूसले युक्रेनमा आक्रमण गरेको ठ्याक्कै मिति निर्दिष्ट गर्दैन। यो प्रतिक्रियाले संकेत गर्दछ कि LLM स्रोत दस्तावेजहरूमा उपलब्ध जानकारी प्रयोग गरिरहेको छ तर पाठमा ठ्याक्कै विवरणको अभावलाई स्वीकार गर्दछ।

प्रतिक्रिया सही छ किनभने यसले सन्दर्भमा रूसको आक्रमणसँग सम्बन्धित घटनाहरू उल्लेख गरिएको छ तर ठ्याक्कै आक्रमणको मिति निर्दिष्ट गरिएको छैन भन्ने कुरा पहिचान गर्दछ। यसले LLM को प्रदान गरिएको जानकारी बुझ्ने क्षमता देखाउँछ र सामग्रीमा रहेको खाली ठाउँलाई चिन्हित गर्दछ। LLM ले प्रयोगकर्तालाई ठ्याक्कै मिति पत्ता लगाउन बाह्य स्रोतहरू वा ऐतिहासिक अभिलेखहरूको खोजी गर्न सुझाव दिन्छ, जसले अपूर्ण जानकारी हुँदा सतर्कता देखाउँछ।

### स्रोत नोडहरूको विश्लेषण
हाइब्रिड खोजको उदाहरणमा स्रोत नोडहरूले रूसको युक्रेनमा कार्यहरूको विरुद्धमा अमेरिकाको प्रतिक्रिया सम्बन्धी भाषणका अंशहरू समावेश गर्दछ। यी नोडहरूले व्यापक भू-राजनीतिक प्रभाव र आक्रमणको प्रतिक्रियामा अमेरिका र यसको सहयोगीहरूले उठाएका कदमहरूलाई जोड दिन्छन्, तर ठ्याक्कै आक्रमणको मिति उल्लेख गर्दैनन्। यो LLM प्रतिक्रियासँग मेल खान्छ, जसले सही रूपमा सन्दर्भमा ठ्याक्कै मिति जानकारीको अभावलाई पहिचान गर्दछ।


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### हाइब्रिड w/रिर्याङ्किङ: LLM प्रतिक्रिया विश्लेषण
हाइब्रिड w/रिर्याङ्किङ उदाहरणमा, LLM प्रतिक्रियाले थप सन्दर्भ प्रदान गर्दछ जसले देखाउँछ कि घटना भाषण दिइनु भन्दा छ दिन अघि भएको थियो। यसले संकेत गर्दछ कि LLM ले भाषणको समयको आधारमा आक्रमणको मिति अनुमान गर्न सक्षम छ, यद्यपि सटीकता सुनिश्चित गर्न भाषणको ठ्याक्कै मिति थाहा हुनु आवश्यक छ।

यो प्रतिक्रिया सन्दर्भ संकेतहरूको प्रयोग गरेर थप जानकारीमूलक उत्तर दिन सक्ने सुधारिएको क्षमता देखाउँछ। यसले रिर्याङ्किङको फाइदा उजागर गर्दछ, जहाँ LLM ले थप सान्दर्भिक जानकारी पहुँच गर्न र प्राथमिकता दिन सक्छ ताकि चाहिएको विवरण (जस्तै, आक्रमणको मिति) को नजिकको अनुमान प्रदान गर्न सकियोस्।

### स्रोत नोडहरूको विश्लेषण
यस उदाहरणमा स्रोत नोडहरूले रूसको आक्रमणको समयको सन्दर्भ समावेश गर्दछ, विशेष गरी उल्लेख गर्दै कि यो भाषण भन्दा छ दिन अघि भएको थियो। यद्यपि ठ्याक्कै मिति स्पष्ट रूपमा उल्लेख गरिएको छैन, नोडहरूले अस्थायी सन्दर्भ प्रदान गर्दछ जसले LLM लाई थप सूक्ष्म प्रतिक्रिया दिन सक्षम बनाउँछ। यो विवरणको समावेशले देखाउँछ कि रिर्याङ्किङले LLM लाई प्रदान गरिएको सन्दर्भबाट जानकारी निकाल्न र अनुमान गर्न सक्ने क्षमता सुधार गर्न कसरी मद्दत गर्दछ, जसले थप सटीक र जानकारीमूलक प्रतिक्रिया दिन्छ।


**नोट:**
यस नोटबुकमा, हामीले NVIDIA API Catalog बाट NVIDIA NIM माइक्रोसर्भिसहरू प्रयोग गरेका छौं।  
माथिका APIहरू, `NVIDIA (llms)`, `NVIDIAEmbedding`, र [Azure AI Search Semantic Hybrid Retrieval (बिल्ट-इन रीर्याङ्किङ)](https://learn.microsoft.com/azure/search/semantic-search-overview)।  
नोट गर्नुहोस्, माथिका APIहरूले स्व-होस्ट गरिएको माइक्रोसर्भिसहरूको पनि समर्थन गर्न सक्छ।  

**उदाहरण:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको हो। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धता हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
